In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import numpy as np
import datetime

In [15]:
df_l_1nf = pd.read_csv('data/clean_df_labels_1nf.csv', index_col=[0])

In [16]:
df_l_1nf

,id,topic_champion
0,1413639045268217856,aatrox
0,1413639045268217856,ahri
1,1414479504945270784,aatrox
1,1414479504945270784,ahri
2,1412812522873233413,aatrox
...,...,...
324830,1503948740775882753,miss fortune
324831,1504077934646157323,miss fortune
324832,1507454234249158657,miss fortune
324833,1508205418438397961,aatrox


In [4]:
twitter_data = pd.read_csv('data/word2Vec.csv',index_col=[0])

In [5]:
twitter_data

,id,text
0,1413639045268217856,@KegsnEggs Wheres league of legends?
1,1414479504945270784,Watched the #LCS games with @CaptainFlowers to...
2,1412812522873233413,Wait why the new league of legends character s...
3,1414155902718988289,Starcraft I and II (2 games?)\nWarcraft III\nC...
4,1412275542280257536,Just called a sweaty neckbeard in league of le...
...,...,...
324829,1500316004370558979,Miss Fortune Highlights\n#LoL #wildrift #missf...
324830,1503948740775882753,eres muy joven \nMISS FORTUNE ES LA COMEDY QU...
324831,1504077934646157323,@hammyinmiami Im still looking for someone to...
324832,1507454234249158657,After spending more than 30 mins on hold waiti...


In [6]:
play_rate = pd.read_csv('data/play_rates_final.csv',index_col=[0])

In [7]:
play_rate

,Date,Champion,Play Rate
0,12/15/2021,AurelionSol,0.001998
1,12/15/2021,Shyvana,0.019980
2,12/15/2021,Olaf,0.011988
3,12/15/2021,Neeko,0.014985
4,12/15/2021,Zoe,0.021978
...,...,...,...
2339,9/1/2021,Nami,0.117882
2340,9/1/2021,Kaisa,0.074925
2341,9/1/2021,Yasuo,0.097902
2342,9/1/2021,LeeSin,0.163836


In [17]:
import gensim
from tqdm import tqdm

In [18]:
model = gensim.models.Word2Vec.load("twitter_models/word2vec_7-window_2-min_15-epochs.model")

In [19]:
avg_vector = list()
cid_list = list()

In [20]:
vocabulary = model.wv.index_to_key

In [21]:
for index, row in tqdm(twitter_data.iterrows()):
    tweet = row['text']
    cid = row['id']
    tokens = gensim.utils.simple_preprocess(tweet)
    
    valid_vectors = list()
    
    for token in tokens:
        if token in vocabulary:
            valid_vectors.append(model.wv[token])
    np_vec = np.mean(valid_vectors, axis=0)
    list_vec = np_vec.tolist()
    cid_list.append(cid)
    avg_vector.append(list_vec)

2197it [00:09, 253.69it/s]C:\Users\mzkan\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\mzkan\Anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
324824it [25:39, 211.06it/s]


In [22]:
r = len(avg_vector)

In [23]:
for i in range(r):
    if type(avg_vector[i]) is float:
        avg_vector[i] = [avg_vector[i]]

In [24]:
i = 0
while i < r:
    if len(avg_vector[i]) < 100:
        avg_vector[i] = [None for n in range(100)]
    i += 1

In [25]:
avg_vector

[[-0.41128164529800415,
  1.7197850942611694,
  -0.7032268643379211,
  -1.6367788314819336,
  -0.7907332181930542,
  0.32970544695854187,
  -1.3939179182052612,
  -1.0454410314559937,
  1.443267583847046,
  1.0836868286132812,
  1.5065515041351318,
  -2.100162982940674,
  2.6169068813323975,
  1.5121984481811523,
  0.2065030336380005,
  1.7619370222091675,
  0.08684393763542175,
  0.9682862758636475,
  -1.9524991512298584,
  2.502394914627075,
  -2.05934739112854,
  -0.36046740412712097,
  -1.2631254196166992,
  -0.5536218881607056,
  2.2312231063842773,
  -0.7288962602615356,
  0.20335808396339417,
  -1.7822132110595703,
  -0.2651446759700775,
  0.17561927437782288,
  -0.15299683809280396,
  0.673241138458252,
  0.3438860774040222,
  -1.6287174224853516,
  0.429273396730423,
  -2.396466016769409,
  -0.3528469204902649,
  1.074733853340149,
  -1.1571147441864014,
  1.7598360776901245,
  2.3871042728424072,
  1.5286967754364014,
  0.4437033534049988,
  1.120342493057251,
  1.49011778831

In [26]:
len(avg_vector)

324824

In [41]:
vdf = pd.DataFrame(avg_vector, index=cid_list)

In [42]:
vdf = vdf.reset_index().rename({'index':'id'}, axis = 'columns')

In [43]:
vdf.dtypes

id      int64
0     float64
1     float64
2     float64
3     float64
       ...   
95    float64
96    float64
97    float64
98    float64
99    float64
Length: 101, dtype: object

In [44]:
temp = twitter_data.drop(columns=['text'])

In [45]:
temp["id"] = temp["id"].astype(str)

In [46]:
twitter_data["id"] = twitter_data["id"].astype(str)

In [50]:
vdf = vdf.set_index("id")
vdf

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
id,,,,,,,,,,,,,,,,,,,,,
1413639045268217856,-0.411282,1.719785,-0.703227,-1.636779,-0.790733,0.329705,-1.393918,-1.045441,1.443268,1.083687,...,-0.905833,1.086132,0.086560,2.384864,-2.158891,0.282443,0.607967,0.883672,0.718481,-0.260302
1414479504945270784,-1.422017,1.363626,0.293707,-1.684406,-0.948104,-0.368762,-0.942424,0.711208,1.548581,1.182046,...,-0.204566,0.862341,0.229205,1.356155,-1.180108,1.588566,0.046494,0.530762,0.653248,0.398482
1412812522873233413,-1.116567,1.366103,-0.334970,-1.470153,-0.314988,-0.475279,-0.584459,0.376210,1.775365,2.088222,...,-0.625561,0.992963,-0.692946,2.738378,-2.601780,1.286406,0.211580,0.194270,0.633419,0.181391
1414155902718988289,-1.314340,1.598936,-0.106428,-1.834226,-0.980251,-0.488489,-1.310665,0.522288,1.393433,1.078314,...,-0.140301,0.657344,0.155634,1.838793,-1.539211,1.509621,-0.035111,0.597226,0.546578,0.058349
1412275542280257536,-0.847964,1.529997,0.180924,-1.988319,-1.757011,-1.043975,-0.616088,0.419742,1.318848,1.559255,...,-0.085001,1.450776,-0.385315,1.558595,-1.219505,1.471561,1.029450,1.001750,0.616875,0.468963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500316004370558979,-0.219251,0.265760,0.740004,-0.158504,0.149798,1.251789,-0.117240,-0.452810,-0.530418,0.460314,...,0.005523,0.849139,1.149406,0.808529,-0.040359,-0.476912,0.484213,0.514124,0.590910,-0.096222
1503948740775882753,-0.120865,-0.489800,1.386335,0.267387,-0.160181,1.087525,0.426099,-1.755171,0.353242,0.158149,...,0.811214,-1.136388,-0.815583,-0.051913,1.374635,1.023732,-0.452398,1.189996,0.147268,-1.319749
1504077934646157323,-0.299722,1.503631,0.702907,-2.149230,-1.254734,-1.435591,-0.354698,0.484389,0.838898,0.933674,...,0.482964,0.689122,-0.455043,1.227979,-0.524243,1.755819,0.349831,1.068570,0.306560,0.940990


In [51]:
twitter_vectors = vdf

In [52]:
twitter_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
id,,,,,,,,,,,,,,,,,,,,,
1413639045268217856,-0.411282,1.719785,-0.703227,-1.636779,-0.790733,0.329705,-1.393918,-1.045441,1.443268,1.083687,...,-0.905833,1.086132,0.086560,2.384864,-2.158891,0.282443,0.607967,0.883672,0.718481,-0.260302
1414479504945270784,-1.422017,1.363626,0.293707,-1.684406,-0.948104,-0.368762,-0.942424,0.711208,1.548581,1.182046,...,-0.204566,0.862341,0.229205,1.356155,-1.180108,1.588566,0.046494,0.530762,0.653248,0.398482
1412812522873233413,-1.116567,1.366103,-0.334970,-1.470153,-0.314988,-0.475279,-0.584459,0.376210,1.775365,2.088222,...,-0.625561,0.992963,-0.692946,2.738378,-2.601780,1.286406,0.211580,0.194270,0.633419,0.181391
1414155902718988289,-1.314340,1.598936,-0.106428,-1.834226,-0.980251,-0.488489,-1.310665,0.522288,1.393433,1.078314,...,-0.140301,0.657344,0.155634,1.838793,-1.539211,1.509621,-0.035111,0.597226,0.546578,0.058349
1412275542280257536,-0.847964,1.529997,0.180924,-1.988319,-1.757011,-1.043975,-0.616088,0.419742,1.318848,1.559255,...,-0.085001,1.450776,-0.385315,1.558595,-1.219505,1.471561,1.029450,1.001750,0.616875,0.468963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500316004370558979,-0.219251,0.265760,0.740004,-0.158504,0.149798,1.251789,-0.117240,-0.452810,-0.530418,0.460314,...,0.005523,0.849139,1.149406,0.808529,-0.040359,-0.476912,0.484213,0.514124,0.590910,-0.096222
1503948740775882753,-0.120865,-0.489800,1.386335,0.267387,-0.160181,1.087525,0.426099,-1.755171,0.353242,0.158149,...,0.811214,-1.136388,-0.815583,-0.051913,1.374635,1.023732,-0.452398,1.189996,0.147268,-1.319749
1504077934646157323,-0.299722,1.503631,0.702907,-2.149230,-1.254734,-1.435591,-0.354698,0.484389,0.838898,0.933674,...,0.482964,0.689122,-0.455043,1.227979,-0.524243,1.755819,0.349831,1.068570,0.306560,0.940990


In [53]:
twitter_vectors.to_csv('data/twitter_vectors.csv')